In [1]:
import openpyxl as xl
from openpyxl.utils.cell import *

In [2]:
class Element:   #Define a class for each of the elements from each line of the BEATCH file 

    def __init__(self, line): #Initialise the class
        self.ProcessRawLine(line)
        
    def ProcessRawLine(self, line): #Reads the line and takes the relevant part of the string into the rigt variable. Cast to right type as well
        
        self.elemNumber = int( line[0:4] )
        self.elemName   = str( line[4:15] )
        self.elemID     = int( line[36:38] )
        self.elemLength = float( line[38:46] )
        self.elemX      = float( line[62:73] ) 
        self.elemY      = float( line[73:84] ) 
        self.elemZ      = float( line[84:95] ) 
        self.elemHAngle = float( line[94:104] )
        self.elemVAngle = float( line[104:114] )
        
        
        
def ReadBeatchOut(file): #Read a beatch file. Input is filename. Offsets are arbitrary displacements one can add to facilitate plotting
                                                            #Returns a list of "element" objects containing all the useful info.
        
    #Line in BEATCH output before the useful data is. Use this to tell when the interesting data starts.
    
    headerLine = '                                             M  RAD     DEG       M           M           M               RAD           M\n'

    fileElements = []
    with open(file) as f:
        StartRead = False #Flag set to True when we reach headerLine. Tells when we are at the useful lines in the file
        
        for line in f:

            if StartRead: #When we reach the flag start to create objects and add to the list of elements
                fileElements.append(Element(line))
                continue

            if line == headerLine:
                StartRead = True
                next(f) #There is a blank line between header and data. Skips it
    return(fileElements)



def col2num(col):
    num = 0
    for c in col:
        if c in string.ascii_letters:
            num = num * 26 + (ord(c.upper()) - ord('A')) + 1
    return num


def RangeValue(sheet, a, b, c, d):
    return(tuple(tuple(sheet.cell(row=a+i, column=c+j).value for j in range(d-c+1)) for i in range(b-a+1)))


Available Repere:

Repere EHN1-2704 (MBPL.022632.E)             
Repere TCC2-2701 (point P)                   
Repere EHN1-2703 (QNL.022423.E)              
Repere TT81-TT82-TT83-2702 (QNL.022215.E)    
Repere TCC8-2705 (T10.CENTR)                
Repere ECN3-2703 (LAV.8.)                    
Repere EHN2-2707 (QPL.065095.E) 

In [4]:
#filename = "H2-VLE-Budget-BEATCH_v2_1.out"
filename = "H2-2023-Budget-BEATCH_v2_1.out"
beatchoutInput   = r"FinalBeatches/" + filename
Repere = r"Repere EHN1-2703 (QNL.022423.E)"
saveName = r"FinalRepere/" + filename

outfileCAD = r"FinalRepere/" +filename[:-4] + ".CAD"

spreadsheetInput = "Matrices_transformations_zone_nord_template.xlsx"
elements = ReadBeatchOut(beatchoutInput)
wb       = xl.load_workbook(spreadsheetInput) 
sheet = wb["Sheet1"]
#sheet.formula_attributes = {} #Resets which are array formulas


#File stuff, only change if template is changed


startRow = 4 
startColumnLetter = 'L'
startColumnLetterTransposed = 'U'

startColumn = column_index_from_string(startColumnLetter)
startColumnTransposed = column_index_from_string(startColumnLetterTransposed) 





RepereCoordinates = {
"Repere EHN1-2704 (MBPL.022632.E)"            : ["$C$14:$E$16", "$I$14:$I$16"],
"Repere TCC2-2701 (point P)"                  : ["$C$21:$E$23", "$I$21:$I$23"],
"Repere EHN1-2703 (QNL.022423.E)"             : ["$C$28:$E$30", "$I$28:$I$30"],
"Repere TT81-TT82-TT83-2702 (QNL.022215.E)"   : ["$C$35:$E$37", "$I$35:$I$37"],
"Repere TCC8-2705 (T10.CENTR)"                : ["$C$42:$E$44", "$I$42:$I$44"],
"Repere ECN3-2703 (LAV.8.)"                   : ["$C$49:$E$51", "$I$49:$I$51"], 
"Repere EHN2-2707 (QPL.065095.E)"             : ["$C$56:$E$58", "$I$56:$I$58"]
    
}




startRepereMat, endRepereMat = RepereCoordinates[Repere][0].replace('$','').split(':')
startRepereCol = "".join([ch for ch in startRepereMat if ch.isalpha()])
endRepereCol   = "".join([ch for ch in endRepereMat if ch.isalpha()])
startRepereRow = "".join([ch for ch in startRepereMat if not ch.isalpha()])
endRepereRow   = "".join([ch for ch in endRepereMat if not ch.isalpha()])


startRepereMatOffset, endRepereMatOffset = RepereCoordinates[Repere][1].replace('$','').split(':')
startRepereColOffset = "".join([ch for ch in startRepereMatOffset if ch.isalpha()])
endRepereColOffset   = "".join([ch for ch in endRepereMatOffset if ch.isalpha()])
startRepereRowOffset = "".join([ch for ch in startRepereMatOffset if not ch.isalpha()])
endRepereRowOffset   = "".join([ch for ch in endRepereMatOffset if not ch.isalpha()])

#A7 B8
#78 12
RepereMatrix   = RangeValue(sheet,int(startRepereRow), int(endRepereRow), column_index_from_string(startRepereCol), column_index_from_string( endRepereCol))
RepereOffset   = RangeValue(sheet,int(startRepereRowOffset), int(endRepereRowOffset), column_index_from_string(startRepereColOffset), column_index_from_string( endRepereColOffset))
with open(outfileCAD, 'w') as f:
    for i, entry in enumerate(elements):

        #Write the data for the old coordinates
        sheet.cell(row = startRow + i, column = startColumn    ).value = entry.elemName
        sheet.cell(row = startRow + i, column = startColumn + 1).value = entry.elemX
        sheet.cell(row = startRow + i, column = startColumn + 2).value = entry.elemY
        sheet.cell(row = startRow + i, column = startColumn + 3).value = entry.elemZ


        #Write the data for the transposed coordinates
        sheet.cell(row = 2, column = startColumnTransposed + i).value = entry.elemX
        sheet.cell(row = 3, column = startColumnTransposed + i).value = entry.elemY
        sheet.cell(row = 4, column = startColumnTransposed + i).value = entry.elemZ
        
        
        s = ''
        for k, lineMat in enumerate(RepereMatrix):
            
            newCoord = lineMat[0]*entry.elemX + lineMat[1]*entry.elemY + lineMat[2]*entry.elemZ  + RepereOffset[k][0]
            s += str(round(newCoord,5)).rjust(13) + ';'
        
        s += entry.elemName.rjust(11) + '\n'
        f.write(s)
        #print(s[:-1])
    
    
    


transposedRange =  "$" + startColumnLetterTransposed + "$2:$" + get_column_letter(startColumnTransposed + i) + "$4"
    
#for k in range(i):
    #Write the data for the new coordinates
    #sheet.cell(row = startRow + k, column = startColumn + 5).value = '=TRANSPOSE(MMULT(' + RepereCoordinates[Repere][0] + "," + transposedRange + ")+" + RepereCoordinates[Repere][1] + ")"
    #sheet.cell(row = startRow + k, column = startColumn + 6).value = '=TRANSPOSE(MMULT(' + RepereCoordinates[Repere][0] + "," + transposedRange + ")+" + RepereCoordinates[Repere][1] + ")"
    #sheet.cell(row = startRow + k, column = startColumn + 7).value = '=TRANSPOSE(MMULT(' + RepereCoordinates[Repere][0] + "," + transposedRange + ")+" + RepereCoordinates[Repere][1] + ")"

sheet.cell(row = startRow , column = startColumn + 5).value = '=TRANSPOSE(MMULT(' + RepereCoordinates[Repere][0] + "," + transposedRange + ")+" + RepereCoordinates[Repere][1] + ")"
    
    
formulaRange = get_column_letter(startColumn + 5) + str(startRow) + ':' + get_column_letter(startColumn + 7)  + str(startRow + i)
sheet.formula_attributes[get_column_letter(startColumn + 5) +str(startRow) ] = {'t': 'array', 'ref': formulaRange}
    
wb.save(filename = saveName + ".xlsx")
wb.close()

print("Coordinates changed and files written")


Coordinates changed and files written
